Long-term create a jupyter kernel for any REPL interface

https://github.com/n-riesco/nel/blob/master/lib/nel.js

https://github.com/n-riesco/jp-kernel/blob/master/lib/jp-kernel.js

https://github.com/takluyver/bash_kernel/blob/master/bash_kernel/kernel.py

https://github.com/n-riesco/ijavascript/blob/d459956f76a22f9ec89937e02645ee7555f92d2b/lib/kernel.js

https://github.com/notablemind/jupyter-nodejs/blob/1388e1957792ef167ccfc7a9cf6c64c3950ded76/lib/kernel.js

https://github.com/JuliaLang/IJulia.jl

https://github.com/n-riesco/jp-babel/blob/master/lib/kernel.js

looks like it is just passed to the command line in rust

https://github.com/google/evcxr/blob/602db3ef52bfddeb34608877bd72b9d0d112fa26/evcxr/src/module.rs


Fix ijavascript not showing filename

https://nodejs.org/api/vm.html#vm_class_vm_script

Fix the problem with ijavascript not supporting %%

Go through this list and find demo code for every language and make sure it works with our REPL, minimize dependencies

https://github.com/jupyter/jupyter/wiki/Jupyter-kernels

https://jupyter-client.readthedocs.io/en/stable/kernels.html

https://jupyter-client.readthedocs.io/en/stable/wrapperkernels.html


Add HTML, CSS/SCSS kernels with some sort of visual output specially make for the file type

Create a kernels notebook




TODO: implement both meta kernels in python and socket based kernels (through library include in another language or through command like repl interface), translate zmq to every language

TODO: use syntax highlighter to "train" a transpiler to write socket based kernels in any language, at least 
BROWSER C# GO JAVA NODE.JS PHP PYTHON RUBY from Google


In [1]:
var importer = require('../Core');
var setupSockets = importer.import('bind to jupyter zmq sockets');

var PACKAGE_VERSION = require('../package.json').version;

function getVersion(str) {
    return str.split('.').map(v => parseInt(v, 10)).join('.')
}

var kernelJsonTemplate = (kernel_json) => ({
    argv,
    display_name,
    language,
    metadata,
    env,
    interrupt_mode
} = kernel_json)

var kernelMetaTemplate = (meta_kernel) => ({
    do_execute,
    do_complete,
    do_inspect,
    do_history,
    do_is_complete,
    do_shutdown
} = meta_kernel)

var languageTemplate = (language_info) => ({
    mimetype,
    name,
    file_extension,
    version,
    pygments_lexer,
    codemirror_mode,
    nbconvert_exporter
} = language_info)

var kernelInfoTemplate = (kernel_info) => ({
    protocol_version,
    implementation,
    implementation_version,
    banner,
    language_info,
    help_links
} = kernel_info)

var kernelSessionTemplate = (session) => ({
    execute_request,
    inspect_request,
    complete_request,
    history_request,
    kernel_info_request,
    is_complete_request,
    connect_request,
    comm_info_request,
    kernel_info_request,
    shutdown_request,
    interrupt_request
} = session)

/*
var transpile = require("@babel/core").transform.bind({
    presets: [
        [require.resolve("@babel/preset-env"), {
            loose: true,
            targets: {node: true},
        }],
    ],
});
*/

var kernelTemplate = ({
    kernel_info,
    meta_kernel,
    language_info,
    session,
    init,
    bind_sockets,
    install
}) => ({
    ...kernelInfoTemplate({...kernel_info, language_info}),
    ...kernelMetaTemplate(meta_kernel),
    language_info: languageTemplate(language_info),
    session: kernelSessionTemplate(session),
    init,
    bind_sockets,
    install
})

function javascriptTemplate(config) {
    return kernelTemplate({
        kernel_info: {
            protocol_version: 5.1,
            implementation: 'javascript',
            implementation_version: PACKAGE_VERSION,
            banner: 'NodeJS',
            // TODO: automatically create this from installation intructions
            help_links: ['https://nodejs.org']
        },
        language_info: {
            name: 'javascript',
            version: getVersion(process.versions.node),
            file_extension: '.js',
            mimetype: 'application/javascript'
        },
        session: {
            shutdown_request: (request, session) => (session.kill('SIGTERM') && 
                                                     !request.content.restart || session.init()),
            kernel_info_request: () => ({kernel_info_reply: kernelInfoTemplate(kernel_info)})
        },
        meta_kernel: {
            do_execute: (request) => new vm.Script(transpile(request.content.code)),
            do_shutdown: () => {}
        },
        init: (session) => require("child_process").spawn(process.argv[0],
                                                          ['--eval', meta_kernel],
                                                          {cwd: config.cwd}),
        bind_sockets: setupSockets,
        install: () => require('fs').writeFileSync(process.argv[0], JSON.stringify({
            argv: ['python', '-m', 'node', '-f', '{connection_file}'],
            display_name: 'Node JS',
            language: 'javascript'
        }, null, 4))
    })
}

console.log(javascriptTemplate());



reading notebook /Users/briancullinan/jupyter_ops/Core/files.ipynb
compiling /Users/briancullinan/jupyter_ops/Core/files.ipynb[0] aka list project files
compiling /Users/briancullinan/jupyter_ops/Core/files.ipynb[1] aka common ignore paths
reading notebook /Users/briancullinan/jupyter_ops/Analytics/budgeting.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/documentation.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/movie database.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/reconcile timeline data.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/resume.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/socializing.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Art/virtual worlds.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/aws.ipynb
reading notebook /Users/briancullinan/jupyte

reading notebook /Users/briancullinan/jupyter_ops/Selenium/service auth.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Selenium/stack overflow.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Selenium/utilities.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Selenium/What is Selenium.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Selenium/you earned it.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/bash.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/child process.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/compress.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/dates.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/diff.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/heartbeat.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/levenshtein.ipynb
reading 

ReferenceError: javascript is not defined

bind to jupyter zmq sockets?


In [ ]:
var zmq = require("jmp").zmq;
var util = require('util');

function setupSockets(config) {
  const sockets = {
    control: {
      port: config.control_port,
      type: 'xrep',
    },
    shell: {
      port: config.shell_port,
      type: 'xrep',
    },
    stdin: {
      port: config.stdin_port,
      type: 'router',
    },
    iopub: {
      port: config.iopub_port,
      type: 'pub',
    },
    heartbeat: {
      port: config.hb_port,
      type: 'rep',
    }
  }
  return Promise.all(sockets.map(s => setupSocket.apply(null, s, config)))
}

function setupSocket(config, general, done) {
  const sock = zmq.socket(config.type);
  const addr = general.transport + '://' + general.ip + ':' + config.port
  return util.promisify(sock.bind.bind(sock))(addr)
}

module.exports = setupSockets;
